In [12]:
%matplotlib notebook

from ipywidgets import *
from srcs.extractData import extractData
from srcs.normalise import normaliseDf
from srcs.estimatedPrice import estimatedPrice
from srcs.trainModel import trainModel


In [13]:
df = extractData('data/data.csv')
df

,km,price
0,240000.0,3650.0
1,139800.0,3800.0
2,150500.0,4400.0
3,185530.0,4450.0
4,176000.0,5250.0
5,114800.0,5350.0
6,166800.0,5800.0
7,89000.0,5990.0
8,144500.0,5999.0
9,84000.0,6200.0


In [14]:
normedDf = normaliseDf(df)
normedDf

,km,price
0,2.637603,-2.032224
1,0.735345,-1.918558
2,0.938480,-1.463894
3,1.603511,-1.426005
4,1.422588,-0.819785
5,0.260730,-0.744008
6,1.247930,-0.403010
7,-0.229073,-0.259033
8,0.824573,-0.252213
9,-0.323996,-0.099900


In [15]:
print(estimatedPrice(1, thetas=(1, 2)))

3


In [27]:
thetas = trainModel(normedDf, plot=False, learningRate=0.1, epoch=1000)
print(thetas)

[1.892005071132038e-16, -0.8561394207905018]


In [28]:
cpDf = normedDf.copy(deep=True)
cpDf['ax+b'] = cpDf['km'].apply(lambda x: thetas[0] + x * thetas[1])
cpDf

,km,price,ax+b
0,2.637603,-2.032224,-2.258156
1,0.735345,-1.918558,-0.629558
2,0.938480,-1.463894,-0.803470
3,1.603511,-1.426005,-1.372829
4,1.422588,-0.819785,-1.217934
5,0.260730,-0.744008,-0.223221
6,1.247930,-0.403010,-1.068402
7,-0.229073,-0.259033,0.196118
8,0.824573,-0.252213,-0.705949
9,-0.323996,-0.099900,0.277386


In [29]:
testDf = cpDf.copy(deep=True)
testDf['price'] = testDf['price'].apply(lambda x: x * (df['price'].std()) + df['price'].mean())
testDf['ax+b'] = testDf['ax+b'].apply(lambda x: x * (df['price'].std()) + df['price'].mean())
testDf['km'] = testDf['km'].apply(lambda x: x * (df['km'].std()) + df['km'].mean())
testDf

,km,price,ax+b
0,240000.0,3650.0,3351.848388
1,139800.0,3800.0,5501.034540
2,150500.0,4400.0,5271.530629
3,185530.0,4450.0,4520.173435
4,176000.0,5250.0,4724.582058
5,114800.0,5350.0,6037.258630
6,166800.0,5800.0,4921.912523
7,89000.0,5990.0,6590.641890
8,144500.0,5999.0,5400.224411
9,84000.0,6200.0,6697.886708


In [34]:
t1 = thetas[1] * (df['price'].std() / df['km'].std())
t0 = df['price'].mean() + df['price'].std() * (thetas[0] - (thetas[1] * (df['km'].mean() / df['km'].std())))
testDf['unnorm'] = testDf['km'].apply(lambda x: t0 + x * t1)
print(t0, t1)
print(thetas)
testDf

8499.599649933214 -0.02144896359170229
[1.892005071132038e-16, -0.8561394207905018]


,km,price,ax+b,unnorm
0,240000.0,3650.0,3351.848388,3351.848388
1,139800.0,3800.0,5501.034540,5501.034540
2,150500.0,4400.0,5271.530629,5271.530629
3,185530.0,4450.0,4520.173435,4520.173435
4,176000.0,5250.0,4724.582058,4724.582058
5,114800.0,5350.0,6037.258630,6037.258630
6,166800.0,5800.0,4921.912523,4921.912523
7,89000.0,5990.0,6590.641890,6590.641890
8,144500.0,5999.0,5400.224411,5400.224411
9,84000.0,6200.0,6697.886708,6697.886708


In [26]:
test2 = df.copy(deep=True)
test2['un'] = test2['km'].apply(lambda x: t0 + x * t1)
test2

,km,price,un
0,240000.0,3650.0,-2.711476e+08
1,139800.0,3800.0,-1.579408e+08
2,150500.0,4400.0,-1.700298e+08
3,185530.0,4450.0,-2.096069e+08
4,176000.0,5250.0,-1.988399e+08
5,114800.0,5350.0,-1.296956e+08
6,166800.0,5800.0,-1.884456e+08
7,89000.0,5990.0,-1.005466e+08
8,144500.0,5999.0,-1.632509e+08
9,84000.0,6200.0,-9.489754e+07


In [21]:
print(f'std {df["price"].std()} mean {df["price"].mean()}')
print(f'std {df["km"].std()} mean {df["km"].mean()}')


std 1319.6541927360352 mean 6331.833333333333
std 52674.24560550151 mean 101066.25


In [22]:
t1 = thetas[1] * (df['price'].std() / df['km'].std())
t0 = df['price'].mean() + df['price'].std() * (thetas[0] - (thetas[1] * (df['km'].mean() / df['km'].std())))
print(thetas)
print(t0, t1)

[2.2435756955966697e-16, -0.8561394207905022]
8499.599649933216 -0.0214489635917023
